In [ ]:
import pandas as pd
import numpy as np
import os
import dpsim

In [ ]:
import requests
url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/-/raw/master/Matpower/case9results.mat'
with open('case9results.mat', 'wb') as out_file:
    content = requests.get(url, stream=True).content
    out_file.write(content)

In [ ]:
mpc_reader = dpsim.matpower.Reader('case9results.mat', 'case9results')

In [ ]:
system = mpc_reader.load_mpc()

In [ ]:
sim_name = 'case9'
dpsim.Logger.set_log_dir('logs/' + sim_name)
logger = dpsim.Logger(sim_name)

for node in system.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)
    
# Parametrize and run simulation
sim = dpsim.Simulation(sim_name, dpsim.LogLevel.info)
sim.set_system(system)
sim.set_time_step(1)
sim.set_final_time(1)
sim.set_domain(dpsim.Domain.SP)
sim.set_solver(dpsim.Solver.NRP)
sim.do_init_from_nodes_and_terminals(False)
sim.add_logger(logger)
sim.run()

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts

log_file = 'logs/' + sim_name +'/' + sim_name + '.csv'
print(log_file)
ts_dpsimpy = rt.read_timeseries_dpsim(log_file)

In [ ]:
results = pd.DataFrame(columns=['Bus', 'V_mag(pu)', 'V_angle(deg)', 'P(MW)', 'Q (MVAr)'])

for i in range(len(system.nodes)):
    node = system.nodes[i].name()
    node_baseV = mpc_reader.mpc_bus_data.loc[mpc_reader.mpc_bus_data['bus_i'] == int(node), 'baseKV'].iloc[0] * 1e3
    w_mw = 1e-6
    results.loc[i] = ([node] + [np.absolute(ts_dpsimpy[node + '.V'].values[-1]) / node_baseV]
        + [np.degrees(np.angle(ts_dpsimpy[node + '.V'].values[-1]))] 
        + [w_mw * np.real(ts_dpsimpy[node + '.S'].values[-1])] 
        + [w_mw * np.imag(ts_dpsimpy[node + '.S'].values[-1])])

In [ ]:
results

In [ ]:
import scipy.io
input_mat = 'case9results.mat'
data = scipy.io.loadmat(input_mat)
 
bus_data = pd.DataFrame(data['case9results'][0][0]['bus'])
bus_data.columns = ["bus_i", "type", "Pd", "Qd", "Gs", "Bs", "area", "Vm", "Va", "baseKV", "zone", "Vmax", "Vmin"]

In [ ]:
bus_data

In [ ]:
assert all(bus_data['Vm'] - results['V_mag(pu)'] < 0.001)

In [ ]:
assert all(bus_data['Va'] - results['V_angle(deg)'] < 0.05)